In [ ]:
%matplotlib inline
%pylab inline

import menpo.io as mio
from menpo.feature import no_op, fast_dsift
from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images
from menpo.image import Image

## Define MCCF patch expert

Start by defining `centralize` and `normalize_norm` feature functions:

In [ ]:
from menpo.feature import ndfeature

@ndfeature
def centralize(x, channels=True):
    if channels:
        m = np.mean(x, axis=(-2, -1))[..., None, None]
    else:
        m = np.mean(x)
    
    x = x - m
    
    return x


@ndfeature
def normalize_norm(x, channels=True):
    x = centralize(x, channels=channels)
    
    if channels:
        norm = np.asarray(np.linalg.norm(x, axis=(-2, -1))[..., None, None])
    else:
        norm = np.asarray(np.linalg.norm(x))
        
    if np.any(norm == 0):
        raise ValueError("Image has 0 variance - can't be normalized")
    else:
        x = x / norm
    
    return x

Multi-Channel Correlation Filters (MCCF) patch expert:

In [ ]:
from copy import deepcopy
from pyfftw.interfaces.numpy_fft import fft2, fftshift
from menpo.image import Image
from menpofit.math.correlationfilter import mccf
from menpofit.math.fft_utils import fft_convolve2d_sum

class MCCFPatchExpert(object):
    r"""
    Multi-Channel Correlation Filter patch expert
    """
    def __init__(self, l=1, normalize_callable=normalize_norm,
                 cosine_mask=True, boundary='constant'):
        self.l = l
        self.normalize_callable = normalize_callable
        self.cosine_mask = cosine_mask
        self.boundary = boundary

    def train(self, X, t):
        # number of samples, number of channels, height and width
        n, k, h, w = X.shape

        if self.cosine_mask:
            # compute cosine mask if required
            cy = np.hanning(h)
            cx = np.hanning(w)
            self.cosine_mask = cy[..., None].dot(cx[None, ...])

        # for each sample
        keep = []
        count = 0
        X = deepcopy(X)
        for j, x in enumerate(X):
            try:
                # normalize it if required
                x = self.normalize_callable(x)
                # apply cosine mask if required
                if self.cosine_mask is not None:
                    x = self.cosine_mask * x
                X[count] = x
                keep.append(j)
                count += 1
            except:
                pass
            
        X = X[:count]

        # compute correlation filter
        self.filter = mccf(X, t, l=self.l, boundary=self.boundary)[0]
        self.rescaled_filter = Image(self.filter).rescale(0.5).pixels
        
        return self

    @property
    def spatial_filter_image(self):
        return Image(self.filter[:, ::-1, ::-1])

    @property
    def frequency_filter_image(self):
        return Image(np.abs(fftshift(fft2(self.filter[:, ::-1, ::-1]))))
    
    def predict(self, x):
        # normalize if required
        x = self.normalize_callable(x)
        # compute filter response
        return fft_convolve2d_sum(x, self.filter)

## Load images

In [ ]:
images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/trainset/', 
                           verbose=True, max_images=100):
    if i.n_channels == 3:
        i = i.crop_to_landmarks_proportion(0.8)
        i = i.rescale_landmarks_to_diagonal_range(200)
        labeller(i, 'PTS', ibug_face_66)
        images.append(i)

In [ ]:
visualize_images(images)

In [ ]:
shapes = [i.landmarks['ibug_face_66'].lms for i in images]

## Train a MCCF patch expert

In [ ]:
from scipy.stats import multivariate_normal
from menpo.shape import PointCloud
from menpofit.base import build_grid
from menpofit.clm.patch_experts import check_context_size

patch_size = (32, 32) 
context_size= 4
sample_offsets = PointCloud([[0, 0], [0, 2], [2, 0], [2, 2]])
covariance = 5
landmark = 42

# check parameters
context_size = check_context_size(context_size, patch_size)

# build desired response
grid = build_grid(patch_size)
response = multivariate_normal(mean=np.zeros(2), cov=covariance).pdf(grid)

samples = []
for (i, s) in zip(images, shapes):
    # choose appropriate landmark
    centre = PointCloud([s.points[landmark]])
    # extract positive sample
    p = i.extract_patches(centre, patch_size=context_size,
                          sample_offsets=sample_offsets,
                          as_single_array=True)[0]

    # add positive sample to list
    samples.append(p.reshape((-1,) + p.shape[-2:]))

# turn list into ndarray
samples = np.asarray(samples)
# train patch experts
patch_expert = MCCFPatchExpert(l=10, cosine_mask=None).train(samples, response[None])

Visualize ideal response:

In [ ]:
Image(response).view(cmap_name='jet')

Visualize samples:

In [ ]:
samples_images = [Image(s) for s in samples]

visualize_images(samples_images)

Visualize filters:

In [ ]:
Image(patch_expert.filter).view_widget()

Visualize response on training patches:

In [ ]:
response_maps = [patch_expert.predict(Image(s)) for s in samples]

visualize_images(response_maps)

Visualize responses on entire training images:

In [ ]:
# response_maps = [patch_expert.predict(i) for i in images]

# visualize_images(response_maps)

In [ ]:
from menpo.shape import PointCloud

landmarks = [PointCloud(s.points[40:42,:])  for s in shapes]

In [ ]:
from menpofit.clm.patch_experts import CorrelationFilterExpertEnsemble
from menpo.shape import PointCloud

cfee = CorrelationFilterExpertEnsemble(
    patch_size=(17, 17),
    context_size=(34, 34),
    response_covariance=1,
    sample_offsets=PointCloud([[0,0],[8,0],[0,8],[8,8],[-8,0],[0,-8],[-8,-8]]))

cfee.train(images, landmarks, verbose=True)

In [ ]:
from menpofit.clm.patch_experts import LinearSVMExpertEnsemble
from menpo.shape import PointCloud

cfee = LinearSVMExpertEnsemble(
    images, 
    landmarks, 
    verbose=True,
    patch_size=(17, 17),
    context_size=(34, 34),
    sample_offsets=PointCloud([[0,0],[8,0],[0,8],[8,8],[-8,0],[0,-8],[-8,-8]]))

cfee.train(images, landmarks, verbose=True)

In [ ]:
visualize_images(cfee.spatial_filter_images)

In [ ]:
responses = []
for (image, landmark) in zip(images, landmarks):
    response = Image(cfee.predict(image, landmark)[1])
    responses.append(response)
    
visualize_images(responses)

In [ ]:
from sklearn import svm
from functools import partial

linear_svm = partial(svm.LinearSVC, class_weight='auto')
linear_svm_fit = linear_svm().fit

class linear_svm(object):
    
    def __init__(self):
        self.svm = svm.LinearSVC(class_weight='auto')
        
    def __call__(self, X, t):
        self.svm.fit(X, t)
        return self.svm.coef_, self.svm.intercept_
    
    

In [ ]:
a = np.array([[2,3],[1,2],[3,4],[7,8],[5,6]])
b = np.array([[1,2],[5,6]])

In [ ]:
for c in b:
    a = np.delete(a, c.reshape((1,2)), axis=0)
    
print a

In [ ]:
from menpofit.base import build_grid

context_size = np.asarray((17, 17))
positive_neighbourhood = np.asarray((1, 1))

grid = build_grid(context_size)
        
positive_mask = np.require(np.zeros(context_size, dtype=np.bool))
half_size = np.floor(context_size / 2)
half_size = np.require(half_size, dtype=int)
start = half_size - np.ceil(positive_neighbourhood - 1)
end = half_size + np.ceil(positive_neighbourhood)
positive_mask[start[0]:end[0], start[1]:end[1]] = True

positive_sample_points = grid[positive_mask]

In [ ]:
negative_sample_step = (4, 4)

negative_mask = np.require(np.zeros(context_size, dtype=np.bool))
negative_mask[::negative_sample_step[0], ::negative_sample_step[1]] = True
negative_mask = np.logical_and(negative_mask, ~positive_mask)

negative_sample_points = grid[negative_mask]

In [ ]:
imshow(negative_mask)